# Process all file in folder





In [1]:
%load_ext autoreload
%autoreload 2

List the files to process in both conditions wt and sega

In [2]:
import pandas as pd

from pathlib import Path

# list files from the src folder
src = Path('/media/cephfs2/jparham/Joe for Jerome /')
dst = Path('./')

filelist = pd.concat((
    pd.DataFrame.from_records(
        [{'path':f, 'name':f.name, 'condition':'sega'}  for d in src.glob('*/*/Crop*') for f in d.glob('[!.]*.tif')]
    ),
    pd.DataFrame.from_records(
    [{'path':f, 'name':f.name, 'condition':'wt'}  for d in src.glob('Div*') for f in d.glob('[!.]*.tif')]
)),axis=0,ignore_index=True)

print(f"Number of files {len(filelist)}")
filelist.head()


Number of files 62


,path,name,condition
0,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_0.tif,sega
1,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_12.tif,sega
2,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_13.tif,sega
3,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_3.tif,sega
4,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_11.tif,sega


In [22]:
import motionquant as mq
filename = filelist.iloc[0]['path']
img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)

In [26]:
df = mq.record("tmp", img, mask, position, speed, diff, flow, rho, div, blob)


(441, 60, 60) (441, 60, 60)


NameError: name 'plt' is not defined

## Process all file

In [29]:
from tqdm.notebook import tqdm
tqdm.pandas()
import motionquant as mq
import dask

results_path = dst/Path('results.h5') # result
if results_path.exists():
    results_path.unlink()

print(f"Saving results in {results_path}")

df = []
for filename in tqdm(filelist['path'].iloc):
    try:
        name = filename.stem        
        img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)
        df.append(mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob))
        mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
    except Exception as e:
        print(f"file '{filename}' could not be processed")
        print(e)
        pass


Saving results in results.h5


0it [00:00, ?it/s]

file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240229_delSegA/CroppedDivisions/01a_delSegA_CM_SyS_reg_Cell_6.tif' could not be processed
No cell found at frame 262


In [30]:
df = pd.concat(df)

In [31]:
df.to_csv(dst/'results.csv')